# Chaines de Markov et épidémiologie :
## propagation d’une épidémie dans une population

### Packages :

In [2]:
# Les packages random et matplotlib sont conseillés.
import numpy as np
import matplotlib.pyplot as plt
import random

## 1 Description du modèle

1. Créez la matrice de transition A, la matrice contenant les probabilités de transition entre les différents états. Vérifiez que cette matrice est stochastique

In [11]:
# A la matrice de transition 
# P(2,2) = 1 ; P(0,1) = 0.08 ; P(0,0) = 0.92 ; P(1,1) = 0.93 ; P(1,2) = 0.07 ; P(0,2) = P(1,0) = P(2,0) = P(2,1) = 0
# Avec: P -> la proba ; 0 -> l'etat S ; 1 -> l'etat I ; 2 -> l'etat R
A = np.array([[0.92,0.08,0],[0,0.93,0.07],[0,0,1]])
print(A)

[[0.92 0.08 0.  ]
 [0.   0.93 0.07]
 [0.   0.   1.  ]]


0.92 + 0.08 + 0 = 0 + 0.93 + 0.07 = 0 + 0 + 1 = 1 

Alors la somme de chaque ligne vaut 1

Donc cette matrice est stochastique

2. Créez Π0 la distribution de probabilité initiale.